In [2]:
import os
import re
from datetime import datetime
from os import path
from typing import List, Dict

from commonroad.common.util import Interval, AngleInterval

from BatchConversion.BatchConverter import BatchConverter, BatchConversionResult
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.EsminiWrapperProvider import EsminiWrapperProvider
from OpenSCENARIO2CR.OpenSCENARIOWrapper.Esmini.StoryBoardElement import EStoryBoardElementType
from OpenSCENARIO2CR.Osc2CrConverter import Osc2CrConverter, EFailureReason
from OpenSCENARIO2CR.util.AbsRel import AbsRel
from OpenSCENARIO2CR.util.ConversionStatistics import EAnalyzer

In [3]:
from os.path import expanduser

esmini_wrapper = EsminiWrapperProvider(preferred_version="v2.26.5").provide_esmini_wrapper()
esmini_wrapper.max_time = 120.0
esmini_wrapper.grace_time = 1.0
esmini_wrapper.ignored_level = EStoryBoardElementType.ACT
esmini_wrapper.log_to_console = False
esmini_wrapper.log_to_file = False

batch_converter = BatchConverter(
    converter_class=Osc2CrConverter,
    sim_wrapper=esmini_wrapper,

    delta_t=0.1,
    goal_state_position_width=5,
    goal_state_position_length=25,
    goal_state_position_use_ego_rotation=True,
    goal_state_time_step=AbsRel[Interval](Interval(-5, 0), is_absolute=False),
    goal_state_orientation=AbsRel[AngleInterval](AngleInterval(-0.8, 0.8), is_absolute=False),

    keep_ego_vehicle=True,
    do_run_cr_monitor=True,
    do_trim_scenario=False,

    ego_filter=re.compile(r".*ego.*", re.IGNORECASE),

    use_implicit_odr_file=True,

    analyzers=[EAnalyzer.STL_MONITOR, EAnalyzer.DRIVABILITY_CHECKER]
)

batch_converter.discover_files(expanduser("~"), re.compile(r".*\.xosc", re.IGNORECASE), recursively=True)

In [4]:
time_conversion_started = datetime.now()
results: Dict[str, BatchConversionResult] = batch_converter.run_batch_conversion(num_worker=0)
time_conversion_finished = datetime.now()

/home/michael/SoftwareProjects/CommonRoad/ASAM-Standards/openscenario-v1.2.0/Examples/CutIn.xosc:   2%|▏         | 4/161 [00:00<00:09, 16.98it/s]                          /home/michael/miniconda3/envs/commonroad2/lib/python3.9/site-packages/commonroad/scenario/lanelet.py:1279: ShapelyDeprecationWarning: STRtree will be changed in 2.0.0 and will not be compatible with versions < 2.
  self._strtee = STRtree(list(self._buffered_polygons.values()))


Hallo: {'plane_conversion': {'error_tolerance': 0.15, 'min_delta_s': 0.5, 'precision': 0.5}, 'file_header': {'author': 'Sebastian Maierhofer', 'affiliation': 'Technical University of Munich', 'source': 'CommonRoad Scenario Designer', 'time_step_size': 0.1}}


/home/michael/SoftwareProjects/CommonRoad/ASAM-Standards/openscenario-v1.2.0/Examples/DoubleLaneChanger.xosc:   2%|▏         | 4/161 [00:00<00:09, 16.98it/s]/home/michael/SoftwareProjects/CommonRoad/commonroad-scenario-designer/crdesigner/map_conversion/opendrive/opendrive_conversion/plane_elements/traffic_signals.py:45: UserWarning: OpenDRIVE/traffic_signals.py: Unknown TrafficSignIDZamunda of ID 605 of subtype 30!
  warnings.warn("OpenDRIVE/traffic_signals.py: Unknown {}"
/home/michael/SoftwareProjects/CommonRoad/commonroad-scenario-designer/crdesigner/map_conversion/opendrive/opendrive_conversion/plane_elements/traffic_signals.py:45: UserWarning: OpenDRIVE/traffic_signals.py: Unknown TrafficSignIDZamunda of ID 334 of subtype -1!
  warnings.warn("OpenDRIVE/traffic_signals.py: Unknown {}"
/home/michael/SoftwareProjects/CommonRoad/commonroad-scenario-designer/crdesigner/map_conversion/opendrive/opendrive_conversion/plane_elements/traffic_signals.py:45: UserWarning: OpenDRIVE/traffic_si

Hallo: {'plane_conversion': {'error_tolerance': 0.15, 'min_delta_s': 0.5, 'precision': 0.5}, 'file_header': {'author': 'Sebastian Maierhofer', 'affiliation': 'Technical University of Munich', 'source': 'CommonRoad Scenario Designer', 'time_step_size': 0.1}}


/home/michael/SoftwareProjects/CommonRoad/ASAM-Standards/openscenario-v1.2.0/Examples/EndOfTrafficJamParameterSet.xosc:   4%|▍         | 7/161 [00:06<02:41,  1.05s/it]/home/michael/SoftwareProjects/CommonRoad/openscenario2cr/OpenSCENARIO2CR/Osc2CrConverter.py:305: UserWarning: <Osc2CrConverter/_pre_parse_scenario> EndOfTrafficJamParameterSet.xosc contains no source file, but references another OpenSCENARIO file: "/home/michael/SoftwareProjects/CommonRoad/ASAM-Standards/openscenario-v1.2.0/Examples/EndOfTrafficJam.xosc"
  warnings.warn(
/home/michael/SoftwareProjects/CommonRoad/ASAM-Standards/openscenario-v1.2.0/Examples/EndofTrafficJamNeighboringLaneOccupied.xosc:   4%|▍         | 7/161 [00:06<02:41,  1.05s/it]

...

/home/michael/SoftwareProjects/CommonRoad/esmini/test/OSC-ALKS-scenarios/Scenarios/ALKS_Scenario_4.6_1_ForwardDetectionRange_TEMPLATE.xosc:  89%|████████▉ | 144/161 [02:09<00:10,  1.69it/s]        

Hallo: {'plane_conversion': {'error_tolerance': 0.15, 'min_delta_s': 0.5, 'precision': 0.5}, 'file_header': {'author': 'Sebastian Maierhofer', 'affiliation': 'Technical University of Munich', 'source': 'CommonRoad Scenario Designer', 'time_step_size': 0.1}}


/home/michael/SoftwareProjects/CommonRoad/esmini/test/OSC-ALKS-scenarios/Scenarios/ALKS_Scenario_4.6_2_LateralDetectionRange_TEMPLATE.xosc:  90%|█████████ | 145/161 [02:09<00:08,  1.79it/s]

Hallo: {'plane_conversion': {'error_tolerance': 0.15, 'min_delta_s': 0.5, 'precision': 0.5}, 'file_header': {'author': 'Sebastian Maierhofer', 'affiliation': 'Technical University of Munich', 'source': 'CommonRoad Scenario Designer', 'time_step_size': 0.1}}


/home/michael/SoftwareProjects/CommonRoad/esmini/test/OSC-ALKS-scenarios/Variations/ALKS_Scenario_4.1_1_FreeDriving_Variation.xosc:  91%|█████████ | 146/161 [02:10<00:07,  1.88it/s]        /home/michael/SoftwareProjects/CommonRoad/openscenario2cr/OpenSCENARIO2CR/Osc2CrConverter.py:305: UserWarning: <Osc2CrConverter/_pre_parse_scenario> ALKS_Scenario_4.1_1_FreeDriving_Variation.xosc contains no source file, but references another OpenSCENARIO file: "/home/michael/SoftwareProjects/CommonRoad/esmini/test/OSC-ALKS-scenarios/Variations/../Scenarios/ALKS_Scenario_4.1_1_FreeDriving_TEMPLATE.xosc"
  warnings.warn(
/home/michael/SoftwareProjects/CommonRoad/esmini/test/OSC-ALKS-scenarios/Variations/ALKS_Scenario_4.1_2_SwervingLeadVehicle_Variation.xosc:  91%|█████████ | 146/161 [02:10<00:07,  1.88it/s]/home/michael/SoftwareProjects/CommonRoad/openscenario2cr/OpenSCENARIO2CR/Osc2CrConverter.py:305: UserWarning: <Osc2CrConverter/_pre_parse_scenario> ALKS_Scenario_4.1_2_SwervingLeadVehicle_Variatio

In [5]:
from commonroad.scenario.scenario import Tag
from commonroad.common.file_writer import CommonRoadFileWriter, OverwriteExistingFile
import yaml

storage_path = "results/{}".format(time_conversion_finished.isoformat(sep="_", timespec="seconds"))
store = True

if store:
    if not path.exists(storage_path):
        os.mkdir(storage_path)
    statistics_dict = {}
    written_scenarios: List[str] = []
    for scenario_path, result in results.items():
        basename = path.basename(scenario_path)
        path_no_extension = path.join(storage_path, path.splitext(basename)[0])
        if path_no_extension in written_scenarios:
            i = 1
            while True:
                if (new_path := path_no_extension + f"-{i}") not in written_scenarios:
                    path_no_extension = new_path
                    break
                else:
                    i += 1
        written_scenarios.append(path_no_extension)

        if result.without_exception:
            if isinstance(result.conversion_result, EFailureReason):
                statistics_dict[path_no_extension] = ("failure", result.conversion_result.name)
            else:
                scenario, planning_problem_set, statistics = result.conversion_result
                statistics_dict[path_no_extension] = ("success", statistics.to_dict())
                CommonRoadFileWriter(
                    scenario=scenario,
                    planning_problem_set=planning_problem_set,
                    author="ADD AUTHOR HERE",
                    affiliation="ADD AFFILIATION HERE",
                    source="ADD SOURCE HERE",
                    tags={Tag.SIMULATED},
                ).write_to_file(path_no_extension + ".xml", OverwriteExistingFile.ALWAYS)
        else:
            statistics_dict[path_no_extension] = ("error", result.exception.to_dict())

    with open(path.join(storage_path, "statistics.yml"), "w") as file:
        yaml.safe_dump(statistics_dict, file)


In [6]:

from OpenSCENARIO2CR.ConversionAnalyzer.ErrorAnalysisResult import ErrorAnalysisResult

errors: Dict[ErrorAnalysisResult, int] = {}
for result in results.values():
    if result.without_exception:
        if not isinstance(result.conversion_result, EFailureReason):
            scenario, planning_problem_set, statistics = result.conversion_result
            for analysis_type, analysis_result in statistics.analysis.items():
                for vehicle_name, vehicle_result in analysis_result.items():
                    if isinstance(vehicle_result, ErrorAnalysisResult):
                        if vehicle_result in errors:
                            errors[vehicle_result] += 1
                        else:
                            errors[vehicle_result] = 1

for error, count in errors.items():
    print(f"{count}\n{error.traceback_text}")
    print("\n" * 3)